In [ ]:
import os
import numpy as np
# import working directory to check functions
os.chdir('/Users/schiend/Desktop/DOHERTY/R-workspace/cecelia/inst')

# config
import py.config_utils as cfg

%load_ext autoreload
%autoreload 2

In [537]:
base_dir = '/Volumes/USER_data/Dominik/Experiments/DS_N055/DATA/20230314/XCR1_LN-reverb-10mm-1300nm-1700nm-1'
im_path = os.path.join(base_dir, 'ChanA_001_001_001_001.tif')
zarr_path = base_dir + '.ome.zarr'

In [538]:
# read in OME XML from first image sequence
import tifffile
import py.zarr_utils as zarr_utils
import py.ome_xml_utils as ome_xml_utils
from py.dim_utils import DimUtils

im = tifffile.TiffFile(im_path)
x_array = im.asarray()
omexml = ome_xml_utils.parse_meta(im_path)
dim_utils = DimUtils(omexml)
dim_utils.calc_image_dimensions(x_array.shape)

In [539]:
# split y axis
# TODO this is specific to Thorlabs 3P output
split_size = int(dim_utils.dim_val('Y')/dim_utils.dim_val('X'))

In [739]:
# DEBUG take first x slices
debug_slices = 12
debug_start_slice = 48

In [740]:
# split array along Y
x_split = np.array_split(x_array, split_size, axis = dim_utils.dim_idx('Y'))

In [741]:
x_split = [x[debug_start_slice:(debug_start_slice + debug_slices), :, :, :] for x in x_split]

In [742]:
stack_dim = 'Z'
skip_tiles = 1
nscales = 1
physical_stack_scale = 2

In [743]:
# get or create stack axis
stack_dim_idx = dim_utils.dim_idx(stack_dim)
stack_array = False

# check whether to create this dimension
if stack_dim_idx is None:
    stack_dim_idx = dim_utils.default_order.index(stack_dim)
    stack_array = True

In [744]:
import math

In [745]:
"""
scale intensity
exp[𝑚𝑛(Δ𝑧/ls − ln(𝑠))] 
m - multiphoton number
n - plane index
s - relative incident laser power ratio between planes
d𝑧 - spacing between planes
ls - mean free path
"""
def reverb_intensity_scale(m, n, s, dz, ls, x = None, scale = None):
    # get y 
    y = 1/math.exp(m * n * (dz/ls - math.log(s)))
    
    # scale?
    if scale is not None:
        y = y/scale
    
    if x is not None:
        return (x * y).astype(x.dtype)
    else:
        return y

In [746]:
m = 3
s = 2
dz = 26
ls = 200

In [747]:
reverb_intensity_scale(m, len(x_split_shift[::(skip_tiles + 1)]), s, dz, ls)

860.7173476383321

In [748]:
# create shifts to create image
shifts = np.array([
    [14.74, -2.18],
    [14.74, -2.18],
    [13.41, 16.31],
    [13.41, 16.31],
    [8.92, -4.23],
    [8.92, -4.23]
])

In [749]:
# generate slices for images
shift_slices_target = [[slice(None) for _ in x_array.shape] for _ in range(len(shifts))]
shift_slices_source = [[slice(None) for _ in x_array.shape] for _ in range(len(shifts))]
dim_val = dim_utils.dim_val('X')

for i, shift in enumerate(shifts):
    # create slices for source and target
    if shift[0] > 0:
        shift_slices_target[i][dim_utils.dim_idx('Y')] = slice(round(shift[0]), dim_val, 1)
        shift_slices_source[i][dim_utils.dim_idx('Y')] = slice(0, dim_val - round(shift[0]), 1)
    else:
        shift_slices_target[i][dim_utils.dim_idx('Y')] = slice(0, dim_val + round(shift[0]), 1)
        shift_slices_source[i][dim_utils.dim_idx('Y')] = slice(-round(shift[0]), dim_val, 1)
        
    if shift[1] > 0:
        shift_slices_target[i][dim_utils.dim_idx('X')] = slice(round(shift[1]), dim_val, 1)
        shift_slices_source[i][dim_utils.dim_idx('X')] = slice(0, dim_val - round(shift[1]), 1)
    else:
        shift_slices_target[i][dim_utils.dim_idx('X')] = slice(0, dim_val + round(shift[1]), 1)
        shift_slices_source[i][dim_utils.dim_idx('X')] = slice(-round(shift[1]), dim_val, 1)
    
shift_slices_target = [tuple(x) for x in shift_slices_target]
shift_slices_source = [tuple(x) for x in shift_slices_source]

In [750]:
x_split_shift = x_split[:2]

# generate new image
for i, x in enumerate(x_split[2:]):
    # zero image
    zero_im = np.zeros_like(x)
    
    # copy image
    zero_im[shift_slices_target[i]] = x[shift_slices_source[i]]
    # zero_im[shift_slices_source[i]] = x[shift_slices_target[i]]
    
    x_split_shift.append(zero_im)

In [751]:
 if stack_array is True:
    x_new = np.stack(
        [reverb_intensity_scale(x, m, n + 1, s, dz, ls) \
         for n, x in enumerate(x_split[::(skip_tiles + 1)])],
        axis = stack_dim_idx
    )
else:
    # get max reverb scale
    max_scale = reverb_intensity_scale(m, len(x_split_shift[::(skip_tiles + 1)]), s, dz, ls)
    
    # contact with shift slices
    x_new_list = [reverb_intensity_scale(m, n + 1, s, dz, ls, x = x, scale = max_scale) \
         for n, x in enumerate(x_split_shift[::(skip_tiles + 1)])]
    x_new_list.reverse()
    
    x_new = np.concatenate(x_new_list, axis = stack_dim_idx)

In [752]:
# use median filter
from skimage.filters.rank import median
from skimage.morphology import ball

In [753]:
im_slices = [slice(None) for _ in range(len(x_new.shape))]

for i in range(dim_utils.dim_val('C')):
    im_slices = list(im_slices)
    im_slices[dim_utils.dim_idx('C')] = i
    im_slices = tuple(im_slices)
    
    x_new[im_slices] = median(x_new[im_slices], ball(3))

/Users/schiend/Library/r-miniconda/envs/r-cecelia-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:276: UserWarning: Bad rank filter performance is expected due to a large number of bins (14642), equivalent to an approximate bitdepth of 13.8.
  image, footprint, out, mask, n_bins = _handle_input_3D(
/Users/schiend/Library/r-miniconda/envs/r-cecelia-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:276: UserWarning: Bad rank filter performance is expected due to a large number of bins (4628), equivalent to an approximate bitdepth of 12.2.
  image, footprint, out, mask, n_bins = _handle_input_3D(
/Users/schiend/Library/r-miniconda/envs/r-cecelia-env/lib/python3.9/site-packages/skimage/filters/rank/generic.py:276: UserWarning: Bad rank filter performance is expected due to a large number of bins (2205), equivalent to an approximate bitdepth of 11.1.
  image, footprint, out, mask, n_bins = _handle_input_3D(


In [754]:
zarr_utils.create_multiscales(x_new, zarr_path, nscales = nscales)

(48, 4, 512, 512)
None


/Users/schiend/Library/r-miniconda/envs/r-cecelia-env/lib/python3.9/site-packages/zarr/creation.py:233: UserWarning: ignoring keyword argument 'mode'
  warn('ignoring keyword argument %r' % k)


In [755]:
# create shape dict for new image
shape_dict = {'Y': int(dim_utils.dim_val('Y')/split_size)}
shape_dict[stack_dim.upper()] = x_new.shape[stack_dim_idx]
scale_dict = dict()
scale_dict[stack_dim.upper()] = physical_stack_scale

# change image dimensions in xml
omexml_new = ome_xml_utils.set_im_size_with_dict(omexml, shape_dict)
omexml_new = ome_xml_utils.set_physical_size_with_dict(omexml_new, scale_dict)

# add metadata
ome_xml_utils.write_ome_xml(zarr_path, omexml_new)

In [756]:
from py.napari_utils import NapariUtils

napari_utils = NapariUtils()
napari_utils.viewer = None
napari_utils.open_viewer()

In [757]:
channel_names = [
    'A', 'B', 'C', 'D'
]

In [758]:
napari_utils.open_image(
    zarr_path,
    use_channel_axis = True, as_dask = True,
    channel_names = channel_names
)

[48, 4, 512, 512]


In [759]:
from napari_animation import AnimationWidget
animation_widget = AnimationWidget(napari_utils.viewer)
napari_utils.viewer.window.add_dock_widget(animation_widget, area='right')